In [10]:
import os

BASE_DIR = os.path.dirname(os.getcwd())

BASE_DIR





'/Users/andresfelipecastrosalazar/Desktop/ETL/ETL_Lab_1'

In [11]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

DATA_RAW = os.path.join(BASE_DIR, "data", "raw")
DATA_TRANSFORMED = os.path.join(BASE_DIR, "data", "transformed")
LOG_FILE = os.path.join(BASE_DIR, "logs", "log_file.txt")
DB_PATH = os.path.join(BASE_DIR, "etl_database.db")

os.listdir(DATA_RAW)

['used_car_prices1.csv',
 'used_car_prices2.csv',
 'used_car_prices3.csv',
 'used_car_prices1.json',
 'used_car_prices3.xml',
 'used_car_prices2.xml',
 'used_car_prices3.json',
 'used_car_prices1.xml',
 'used_car_prices2.json']

# Logging

In [12]:
def log_progress(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, "a") as f:
        f.write(f"{timestamp}, {message}\n")


# Extraction: CSV

In [13]:
def extract_from_csv(path):
    files = glob.glob(os.path.join(path, "*.csv"))

    if not files:
        raise FileNotFoundError(f"No CSV files found in {path}")

    df = pd.concat(
        [pd.read_csv(file) for file in files],
        ignore_index=True
    )
    return df



In [14]:
df_csv = extract_from_csv(DATA_RAW)
df_csv.head()


,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel


# Extraction: JSON

In [15]:
def extract_from_json(path):
    files = glob.glob(os.path.join(path, "*.json"))
    df = pd.concat(
        [pd.read_json(file, lines=True) for file in files],
        ignore_index=True
    )
    return df


In [16]:
df_json = extract_from_json(DATA_RAW)
df_json.head()


,car_model,year_of_manufacture,price,fuel
0,ritz,2012,4626.865672,Diesel
1,ritz,2011,3507.462687,Petrol
2,swift,2014,7388.059701,Diesel
3,ertiga,2014,8955.223881,Diesel
4,dzire,2014,8208.955224,Diesel


# Extraction: XML

In [17]:
def extract_from_xml(path):
    files = glob.glob(os.path.join(path, "*.xml"))
    records = []

    for file in files:
        tree = ET.parse(file)
        root = tree.getroot()

        for row in root:
            records.append({
                "car_model": row.find("car_model").text,
                "year_of_manufacture": int(row.find("year_of_manufacture").text),
                "price": float(row.find("price").text),
                "fuel": row.find("fuel").text
            })

    return pd.DataFrame(records)


In [18]:
df_xml = extract_from_xml(DATA_RAW)
df_xml.head()


,car_model,year_of_manufacture,price,fuel
0,corolla altis,2016,21985.074627,Diesel
1,etios gd,2015,7089.552239,Diesel
2,innova,2017,34328.358209,Diesel
3,innova,2015,18656.716418,Diesel
4,innova,2005,5208.955224,Diesel


# Unify datasets

In [19]:
log_progress("Extract phase started")

df_extracted = pd.concat([df_csv, df_json, df_xml], ignore_index=True)

log_progress("Data extraction completed")

df_extracted.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   car_model            90 non-null     object 
 1   year_of_manufacture  90 non-null     int64  
 2   price                90 non-null     float64
 3   fuel                 90 non-null     object 
dtypes: float64(1), int64(1), object(2)
memory usage: 2.9+ KB


# Transform

In [20]:
def transform(df):
    df["price"] = df["price"].round(2)
    return df


In [21]:
log_progress("Transform phase started")

df_transformed = transform(df_extracted)

log_progress("Data transformation completed")

df_transformed.head()


,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel


# Load a CSV

In [22]:
TARGET_FILE = os.path.join(DATA_TRANSFORMED, "transformed_data.csv")

df_transformed.to_csv(TARGET_FILE, index=False)

log_progress("Data saved to CSV")

pd.read_csv(TARGET_FILE).head()


,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.00,Petrol
1,sx4,2013,7089.55,Diesel
2,ciaz,2017,10820.90,Petrol
3,wagon r,2011,4253.73,Petrol
4,swift,2014,6865.67,Diesel


# Load a SQLite

In [23]:
conn = sqlite3.connect(DB_PATH)

df_transformed.to_sql(
    "vehicles",
    conn,
    if_exists="replace",
    index=False
)

log_progress("Data loaded to Database")


# Queries SQL

In [24]:
pd.read_sql("""
SELECT car_model, year_of_manufacture, price
FROM vehicles
WHERE year_of_manufacture > 2015
""", conn)


,car_model,year_of_manufacture,price
0,ciaz,2017,10820.90
1,vitara brezza,2018,13805.97
2,ciaz,2016,13059.70
3,alto 800,2017,4253.73
4,ertiga,2016,11567.16
5,ertiga,2016,11567.16
6,alto k10,2016,4253.73
7,ignis,2017,7313.43
8,swift,2017,8955.22
9,alto k10,2016,4477.61


In [25]:
pd.read_sql("""
SELECT fuel, COUNT(*) AS count
FROM vehicles
GROUP BY fuel
""", conn)


,fuel,count
0,CNG,2
1,Diesel,36
2,Petrol,52


In [26]:
pd.read_sql("""
SELECT ROUND(AVG(price), 2) AS average_price
FROM vehicles
""", conn)


,average_price
0,10739.8


In [27]:
conn.close()
log_progress("ETL Job Ended")
